In [3]:
import pandas as pd
from dataset import MTL_Dataset

In [4]:
data = MTL_Dataset()

In [26]:
a = pd.read_csv(data.mal_sentiment_d, '\t')
print(a)
dic = {'category': a['category'], 'text': a['text']}
newpd = pd.DataFrame(dic)

                                                   text       category
0                   Mammooka ninghal mass aa pwoli item       Positive
1         Waiting for Malayalam movie  For Tamil paiyan  not-malayalam
2        ദളപതി ഫാൻസിന്റെ വക ഒരു ഒന്നൊന്നര വിജയാശംസകൾ...       Positive
3     #Trailer pwolichuuuu ഓണത്തിന് വന്നങ്ങു തകർത്തേ...       Positive
4      Mammoookkaaaa polichadukkiii katta waiting nv 21       Positive
...                                                 ...            ...
1761            Aa ചിരി uff എന്റെ പൊന്നോ ഇക്ക vere ലെവൽ       Positive
1762   Katta katta katta katta waiting....cant wait man       Positive
1763  Arjun reddy bgm poole thonniyathu enniku mathr...  unknown_state
1764             Fahad ikka ithilum polikum en urappayi       Positive
1765  Njan veendum kanan vannu 100 pravashyam enkilu...       Positive

[1766 rows x 2 columns]


In [28]:
newpd.to_csv('Mal_sentiment_dev.tsv', sep='\t', index=False)